In [120]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns 
import ipywidgets as widgets
import ast
import re



In [139]:
def lac():
    
    with open(r"C:\Users\rossa\Desktop\HENRY\PROYECTO INDIVIDUAL\PI_ML_OPS\src\australian_user_reviews.json", "rt", encoding="utf-8") as file:
        return [ast.literal_eval(line.strip()) for line in file]     

datos = lac()
df_user_reviews = pd.DataFrame(datos)
df_user_reviews.head(3) 

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."


In [140]:
print (type(ast.literal_eval))

<class 'function'>


In [141]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple 

In [142]:
# Inicializa una lista vacía para almacenar los datos de la columna "reviews"
datos_reviews = []

# Recorre el DataFrame y la columna "reviews"
for index, fila in df_user_reviews.iterrows():
    for review in fila["reviews"]:
        datos_reviews.append({
            "user_id": fila["user_id"],
            "user_url": fila["user_url"],
            "funny": review.get("funny", ""),  # Accede a la clave "funny" si existe, o utiliza una cadena vacía si no
            "posted": review.get("posted", ""),  # Accede a la clave "posted" si existe, o utiliza una cadena vacía si no
            "last_edited": review.get("last_edited", ""),
            "item_id": review.get("item_id", ""),
            "helpful": review.get("helpful", ""),
            "recommend":review.get("recommend", ""),
            "review":review.get("review", "")
        })

# Convierte la lista de datos en un nuevo DataFrame
nuevo_df_reviews = pd.DataFrame(datos_reviews)



In [143]:
nuevo_df_reviews.head(2)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.


In [144]:
nuevo_df_reviews = nuevo_df_reviews.drop (["funny", "helpful","last_edited"], axis= 1) 

In [145]:
# Contar la cantidad de valores "None" en la columna "recommend"
nuevo_df_reviews['recommend'].isna().sum()



0

In [146]:
# Eliminar los puntos de la columna 'texto' utilizando .str.replace()
nuevo_df_reviews['posted'] = nuevo_df_reviews['posted'].str.replace('.', '')
nuevo_df_reviews['posted'] = nuevo_df_reviews['posted'].str.replace(',', '')
nuevo_df_reviews['posted'] = nuevo_df_reviews['posted'].str.replace('Posted', '')

In [147]:
nuevo_df_reviews.head(2)

,user_id,user_url,posted,item_id,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,November 5 2011,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,July 15 2011,22200,True,It's unique and worth a playthrough.


In [148]:
# Dividir la columna 'posted' en nuevas columnas de día, mes y año
nuevo_df_reviews[['month', 'day', 'year']] = nuevo_df_reviews['posted'].str.split(' ', n=2, expand=True)

nuevo_df_reviews.drop(columns=['month'], inplace=True)

In [149]:
nuevo_df_reviews.head(2)

,user_id,user_url,posted,item_id,recommend,review,day,year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,November 5 2011,1250,True,Simple yet with great replayability. In my opi...,November,5 2011
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,July 15 2011,22200,True,It's unique and worth a playthrough.,July,15 2011


In [150]:
nuevo_df_reviews = nuevo_df_reviews.rename(columns={'day': 'month'})

In [152]:
# Diccionario de correspondencia entre meses y números
meses_a_numeros = {
    'January': '01',
    'February': '02',
    'March': '03',
    'April': '04',
    'May': '05',
    'June': '06',
    'July': '07',
    'August': '08',
    'September': '09',
    'October': '10',
    'November': '11',
    'December': '12'
}


In [153]:
nuevo_df_reviews.columns

Index(['user_id', 'user_url', 'posted', 'item_id', 'recommend', 'review',
       'month', 'year'],
      dtype='object')

In [154]:
# .map() para transformar los nombres de los meses en números de dos dígitos
nuevo_df_reviews['meses_numeros'] = nuevo_df_reviews['month'].map(meses_a_numeros)

nuevo_df_reviews.head(2)

,user_id,user_url,posted,item_id,recommend,review,month,year,meses_numeros
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,November 5 2011,1250,True,Simple yet with great replayability. In my opi...,November,5 2011,11
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,July 15 2011,22200,True,It's unique and worth a playthrough.,July,15 2011,07


In [155]:
# Dividir la columna 'year' en nuevas columnas de día y año
nuevo_df_reviews[['day', 'year2']] = nuevo_df_reviews ['year'].str.split(' ', n=2, expand=True)
# Eliminar la columna 'posted' original si es necesario
nuevo_df_reviews.drop(columns=['year'], inplace=True) 

In [156]:
# ver el valor que mas se repite
nuevo_df_reviews["year2"].mode()

# imputar el valor en la columna year
nuevo_df_reviews ["year2"].fillna(nuevo_df_reviews["year2"].mode()[0], inplace=True)

In [157]:
nuevo_df_reviews.head(2)

,user_id,user_url,posted,item_id,recommend,review,month,meses_numeros,day,year2
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,November 5 2011,1250,True,Simple yet with great replayability. In my opi...,November,11,5,2011
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,July 15 2011,22200,True,It's unique and worth a playthrough.,July,07,15,2011


In [158]:
#  combinar las columnas en una fecha
nuevo_df_reviews['fecha'] = nuevo_df_reviews.apply(lambda row: f"{row['year2']}-{row['meses_numeros']}-{row['day']}", axis=1)

In [159]:
# Usar .str.zfill() para asegurarse de que todos los valores tengan dos dígitos
nuevo_df_reviews['day'] = nuevo_df_reviews['day'].astype(str).str.zfill(2)

In [160]:
nuevo_df_reviews.head(2)

,user_id,user_url,posted,item_id,recommend,review,month,meses_numeros,day,year2,fecha
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,November 5 2011,1250,True,Simple yet with great replayability. In my opi...,November,11,05,2011,2011-11-5
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,July 15 2011,22200,True,It's unique and worth a playthrough.,July,07,15,2011,2011-07-15


In [161]:
#se reemplaza bool por int
nuevo_df_reviews["recommend"] = nuevo_df_reviews["recommend"].astype(str).str.replace("True", "1")
nuevo_df_reviews["recommend"] = nuevo_df_reviews["recommend"].astype(str).str.replace("False", "0")

In [162]:
nuevo_df_reviews.head(3)

,user_id,user_url,posted,item_id,recommend,review,month,meses_numeros,day,year2,fecha
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,November 5 2011,1250,1,Simple yet with great replayability. In my opi...,November,11,05,2011,2011-11-5
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,July 15 2011,22200,1,It's unique and worth a playthrough.,July,07,15,2011,2011-07-15
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,April 21 2011,43110,1,Great atmosphere. The gunplay can be a bit chu...,April,04,21,2011,2011-04-21


In [163]:
print(nuevo_df_reviews.columns)

Index(['user_id', 'user_url', 'posted', 'item_id', 'recommend', 'review',
       'month', 'meses_numeros', 'day', 'year2', 'fecha'],
      dtype='object')


In [164]:
nuevo_df_reviews= nuevo_df_reviews.drop([ 'posted', 'month', 'meses_numeros', 'day', 'year2', 'review','user_url', 'item_id'],axis=1)

In [165]:
nuevo_df_reviews

,user_id,recommend,fecha
0,76561197970982479,1,2011-11-5
1,76561197970982479,1,2011-07-15
2,76561197970982479,1,2011-04-21
3,js41637,1,2014-06-24
4,js41637,1,2013-09-8
...,...,...,...
59300,76561198312638244,1,2014-07-10
59301,76561198312638244,1,2014-07-8
59302,LydiaMorley,1,2014-07-3
59303,LydiaMorley,1,2014-07-20


In [166]:
nuevo_df_reviews["recommend"] = nuevo_df_reviews["recommend"].astype(int)

In [167]:
nuevo_df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59305 non-null  object
 1   recommend  59305 non-null  int32 
 2   fecha      59305 non-null  object
dtypes: int32(1), object(2)
memory usage: 1.1+ MB


In [168]:
# Función para agregar ceros a la izquierda en los días
def format_date(date_str):
    parts = date_str.split('-')
    if len(parts[2]) == 1:
        parts[2] = '0' + parts[2]
    return '-'.join(parts)

# Aplicar la función para formatear la columna 'fecha' en df_original
nuevo_df_reviews['fecha'] = nuevo_df_reviews['fecha'].apply(format_date)

In [169]:
nuevo_df_reviews.head(3)

,user_id,recommend,fecha
0,76561197970982479,1,2011-11-05
1,76561197970982479,1,2011-07-15
2,76561197970982479,1,2011-04-21


In [170]:
# Contar la cantidad de valores "None" en la columna "price"
nuevo_df_reviews["recommend"].isna().sum()


0

In [171]:
# Contar valores duplicados en el DataFrame
nuevo_df_reviews.duplicated().sum()


4340

In [172]:
nuevo_df_reviews.drop_duplicates(inplace=True)

In [173]:
nuevo_df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54965 entries, 0 to 59304
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    54965 non-null  object
 1   recommend  54965 non-null  int32 
 2   fecha      54965 non-null  object
dtypes: int32(1), object(2)
memory usage: 1.5+ MB


In [174]:
porcentaje = 0.5  
df_original = nuevo_df_reviews.sample(frac=porcentaje)
df_original

,user_id,recommend,fecha
41101,76561198065840081,1,2014-02-29
1955,durolan2535,1,2014-01-03
11614,Ember1654,1,2014-07-01
34731,CantTellMeNothin,1,2014-07-22
23497,76561198089365649,0,2015-08-29
...,...,...,...
19355,76561198065940145,1,2013-12-20
41710,76561198080889573,1,2014-08-18
16343,Flatzone2,1,2015-03-23
9086,Jeff_is_Back,1,2015-05-25


Archivo df_original para la función 1 y 2

In [175]:
# Ruta donde se guarda el archivo CSV
ruta_csv = r"C:\Users\rossa\Desktop\ProyectoIndividualML\ProyectoIndividualML\df_original.csv" 

# Guarda el DataFrame en un archivo CSV
df_original.to_csv(ruta_csv, index=False) 

print("DataFrame guardado como archivo CSV en:", ruta_csv)

DataFrame guardado como archivo CSV en: C:\Users\rossa\Desktop\ProyectoIndividualML\ProyectoIndividualML\df_original.csv


In [176]:
# Especifica la ruta completa del archivo CSV
ruta_csv = r"C:\Users\rossa\Desktop\ProyectoIndividualML\ProyectoIndividualML\df_original.csv"

# Carga el archivo CSV en un DataFrame
df_original = pd.read_csv(ruta_csv)

In [177]:
df_original


,user_id,recommend,fecha
0,76561198065840081,1,2014-02-29
1,durolan2535,1,2014-01-03
2,Ember1654,1,2014-07-01
3,CantTellMeNothin,1,2014-07-22
4,76561198089365649,0,2015-08-29
...,...,...,...
27477,76561198065940145,1,2013-12-20
27478,76561198080889573,1,2014-08-18
27479,Flatzone2,1,2015-03-23
27480,Jeff_is_Back,1,2015-05-25


ARCHIVO USER ITEMS

In [2]:
import ast
import pandas as pd
def lac():
    with open(r"C:\Users\rossa\Desktop\HENRY\PROYECTO INDIVIDUAL\PI_ML_OPS\src\australian_users_items.json", "rt", encoding="utf-8") as file:
        return [ast.literal_eval(line.strip()) for line in file]     

datos = lac()
df_user_items = pd.DataFrame(datos)
df_user_items.head(5)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


Preparacion columnas DF para funcion 3

In [3]:
df_items_items = df_user_items[['user_id','user_url','items']]

df_items_items.sample(5)

,user_id,user_url,items
32200,76561198081265170,http://steamcommunity.com/profiles/76561198081...,"[{'item_id': '4000', 'item_name': 'Garry's Mod..."
69631,76561198073943264,http://steamcommunity.com/profiles/76561198073...,"[{'item_id': '400', 'item_name': 'Portal', 'pl..."
78916,WillowEverdeen,http://steamcommunity.com/id/WillowEverdeen,"[{'item_id': '205790', 'item_name': 'Dota 2 Te..."
12486,76561198051065791,http://steamcommunity.com/profiles/76561198051...,"[{'item_id': '4000', 'item_name': 'Garry's Mod..."
68067,76561198072413044,http://steamcommunity.com/profiles/76561198072...,"[{'item_id': '240', 'item_name': 'Counter-Stri..."


In [4]:

# Obtener el primer diccionario de la columna 'items' (fila 0)
primer_diccionario = df_user_items['items'].iloc[0][0]

# Mostrar las claves (keys) del primer diccionario
claves = list(primer_diccionario.keys())
print(claves)

['item_id', 'item_name', 'playtime_forever', 'playtime_2weeks']


In [5]:
# Descomponer la lista en filas separadas
df_extraer = df_user_items.explode('items')

# Resetear índece del dataframe
df_extraer.reset_index(drop=True, inplace=True)

# Creamos las nuevas columnas para cada clave
df_extraer  = pd.concat([df_extraer , pd.json_normalize(df_extraer['items'])], axis=1)

#debería quedarles 4 columnas mas de las que ya tenían y un monton de filas
#luego pueden eliminar estas columnas items y playtime_2weeks

In [6]:
df_extraer.sample(2)

,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever,playtime_2weeks
2486251,gogi7,496,76561197975349318,http://steamcommunity.com/id/gogi7,"{'item_id': '104200', 'item_name': 'BEEP', 'pl...",104200,BEEP,35.0,0.0
4907111,76561198079705058,55,76561198079705058,http://steamcommunity.com/profiles/76561198079...,"{'item_id': '97100', 'item_name': 'Section 8: ...",97100,Section 8: Prejudice,258.0,0.0


In [7]:
columnas_eliminadas = ['playtime_2weeks', 'items']
df_extraer = df_extraer.drop(columns=columnas_eliminadas)

In [8]:
df_extraer.sample(2)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever
2467458,ElWarpCH,273,76561198063533169,http://steamcommunity.com/id/ElWarpCH,4000,Garry's Mod,1385.0
3303617,robboman17,316,76561198037183433,http://steamcommunity.com/id/robboman17,107410,Arma 3,260.0


In [9]:
# Convertir la Serie de resultados en un nuevo DataFrame
df_items_ph = pd.DataFrame(df_extraer)

df_items_ph.sample(5)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever
5093754,MsVen,96,76561198098352421,http://steamcommunity.com/id/MsVen,368360,60 Seconds!,854.0
4916823,76561198080566079,79,76561198080566079,http://steamcommunity.com/profiles/76561198080...,204630,Retro City Rampage™ DX,28.0
4396367,LoveStage,1192,76561198047333498,http://steamcommunity.com/id/LoveStage,34870,Sniper Ghost Warrior 2,0.0
2113908,ostrichcsgo,124,76561198072357248,http://steamcommunity.com/id/ostrichcsgo,234310,March of War,0.0
3400357,anca1ag0n,374,76561198000554821,http://steamcommunity.com/id/anca1ag0n,28050,Deus Ex: Human Revolution,206.0


In [10]:
# Convertir la columna 'playtime_forever' a tipo (float)
df_items_ph ['playtime_forever'] = df_items_ph['playtime_forever'].fillna(0).astype(float)

# Crear una nueva columna con 'playtime_forever' transformada en horas
df_items_ph ['playtime_hours'] = (df_items_ph['playtime_forever'] / 3600)

df_items_ph ['playtime_hours'] = df_items_ph['playtime_hours'].round(2)

df_items_ph.sample(2) 

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_hours
3866906,the_floor,469,76561197971026111,http://steamcommunity.com/id/the_floor,107310,Cthulhu Saves the World,163.0,0.05
3364806,Ringotuna,55,76561198041575570,http://steamcommunity.com/id/Ringotuna,8190,Just Cause 2,78.0,0.02


In [11]:
df_items_ph.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_forever  float64
 7   playtime_hours    float64
dtypes: float64(2), int64(1), object(5)
memory usage: 315.6+ MB


In [15]:
porcentaje = 0.08 
df_items_ph3 = df_items_ph.sample(frac=porcentaje)
df_items_ph3

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_hours
760451,76561198052474621,134,76561198052474621,http://steamcommunity.com/profiles/76561198052...,348540,Divine Slice of Life,291.0,0.08
3419545,76561198062275390,171,76561198062275390,http://steamcommunity.com/profiles/76561198062...,4700,Medieval II: Total War,121.0,0.03
2053801,addicTT,899,76561197978837955,http://steamcommunity.com/id/addicTT,239200,Amnesia: A Machine for Pigs,0.0,0.00
4973773,76561198085102704,9,76561198085102704,http://steamcommunity.com/profiles/76561198085...,233720,Surgeon Simulator,33.0,0.01
3496277,SuddenNight,222,76561198045982709,http://steamcommunity.com/id/SuddenNight,252490,Rust,917.0,0.25
...,...,...,...,...,...,...,...,...
4799999,pezza349,123,76561198072408067,http://steamcommunity.com/id/pezza349,42680,Call of Duty: Modern Warfare 3,30.0,0.01
1725786,76561197982424065,1271,76561197982424065,http://steamcommunity.com/profiles/76561197982...,24790,Command and Conquer 3: Tiberium Wars,0.0,0.00
5003456,76561198087987687,30,76561198087987687,http://steamcommunity.com/profiles/76561198087...,246280,Happy Wars,0.0,0.00
4947571,76561198082929385,16,76561198082929385,http://steamcommunity.com/profiles/76561198082...,25000,Overgrowth,46.0,0.01


In [16]:
df_items_ph3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 413601 entries, 760451 to 2245159
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           413601 non-null  object 
 1   items_count       413601 non-null  int64  
 2   steam_id          413601 non-null  object 
 3   user_url          413601 non-null  object 
 4   item_id           412296 non-null  object 
 5   item_name         412296 non-null  object 
 6   playtime_forever  413601 non-null  float64
 7   playtime_hours    413601 non-null  float64
dtypes: float64(2), int64(1), object(5)
memory usage: 28.4+ MB


In [17]:
# Ruta donde deseas guardar el archivo CSV
ruta_csv = r"C:\Users\rossa\Desktop\ProyectoIndividualML\ProyectoIndividualML\df_items_ph3.csv" 

# Guarda el DataFrame en un archivo CSV -'index=False' evita que se incluya el índice en el archivo CSV
df_items_ph3.to_csv(ruta_csv, index=False)  

print("DataFrame guardado como archivo CSV en:", ruta_csv)

DataFrame guardado como archivo CSV en: C:\Users\rossa\Desktop\ProyectoIndividualML\ProyectoIndividualML\df_items_ph3.csv


In [20]:
# Especifica la ruta completa del archivo CSV
ruta_csv = r"C:\Users\rossa\Desktop\ProyectoIndividualML\ProyectoIndividualML\df_items_ph3.csv" 

# Carga el archivo CSV en un DataFrame
df_items_ph3 = pd.read_csv(ruta_csv)

In [21]:
df_items_ph3.head(2)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_hours
0,76561197965380720,85,76561197965380720,http://steamcommunity.com/profiles/76561197965...,39000.0,Moonbase Alpha,0.0,0.0
1,76561198009026821,245,76561198009026821,http://steamcommunity.com/profiles/76561198009...,220240.0,Far Cry® 3,2174.0,0.6


ARCHIVO STEAM GAME

In [178]:

# Lee el archivo JSON
with open(r"C:\Users\rossa\Desktop\HENRY\PROYECTO INDIVIDUAL\PI_ML_OPS\src\output_steam_games.json") as f:
    data = f.readlines()

# Elimina saltos de línea y concatena en una cadena JSON válida
data_json_str = "[" + ','.join(map(lambda x: x.rstrip(), data)) + "]"

# Crea un DataFrame desde la cadena JSON
data_df = pd.read_json(data_json_str)


In [179]:
data_df.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
0,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,76561197970982479,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",277.0
1,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,js41637,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",888.0


In [180]:
data_df.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'discount_price', 'specs', 'price',
       'early_access', 'id', 'metascore', 'developer', 'user_id', 'steam_id',
       'items', 'items_count'],
      dtype='object')

In [79]:
# Verifica si hay valores nulos en el DataFrame
hay_nulos = data_df.isnull().any().any()

if hay_nulos:
    print("Hay valores nulos en el DataFrame.")
else:
    print("No hay valores nulos en el DataFrame.")

Hay valores nulos en el DataFrame.


Extracción de columnas ["price", "items_count", "user_id"] para trabajar la función 1 

In [181]:
#DF para función 1
# Seleccion de las columnas 
selected_columns = ["price", "items_count", "user_id"]

# Creación de un nuevo DataFrame con las columnas seleccionadas
df_new_game = data_df[selected_columns]

df_new_game

,price,items_count,user_id
0,None,277.0,76561197970982479
1,None,888.0,js41637
2,None,137.0,evcentric
3,None,328.0,Riot-Punch
4,None,541.0,doctr
...,...,...,...
120440,1.99,NaN,None
120441,4.99,NaN,None
120442,1.99,NaN,None
120443,4.99,NaN,None


In [182]:
cantidad_valores_nulos = df_new_game.isna().sum().sum()

print(f"La cantidad de valores nulos en el DataFrame es: {cantidad_valores_nulos}")

La cantidad de valores nulos en el DataFrame es: 153957


In [183]:
# Verifica  duplicados en todo el DataFrame 
hay_duplicados = df_new_game.duplicated().any()

if hay_duplicados:
    print("Se encontraron duplicados en el DataFrame.")
else:
    print("No se encontraron duplicados en el DataFrame.")

Se encontraron duplicados en el DataFrame.


In [184]:
# Crear una copia del DataFrame df_new_game 
df_new_game_copy = df_new_game.copy()

# Convertir los datos en las columnas "price" y "items_count" a tipo float en la copia
df_new_game_copy['price'] = pd.to_numeric(df_new_game_copy['price'], errors='coerce').astype(float)
df_new_game_copy['items_count'] = pd.to_numeric(df_new_game_copy['items_count'], errors='coerce').astype(float)

# Reemplazar "None" por 0 en las columnas convertidas en la copia
df_new_game_copy['price'].fillna(0, inplace=True)
df_new_game_copy['items_count'].fillna(0, inplace=True)

# Reemplazar el DataFrame original con la copia
df_new_game = df_new_game_copy

In [185]:
df_new_game_copy

,price,items_count,user_id
0,0.00,277.0,76561197970982479
1,0.00,888.0,js41637
2,0.00,137.0,evcentric
3,0.00,328.0,Riot-Punch
4,0.00,541.0,doctr
...,...,...,...
120440,1.99,0.0,None
120441,4.99,0.0,None
120442,1.99,0.0,None
120443,4.99,0.0,None


In [186]:
# Contar la cantidad de valores "None" en la columna "price" 
count_none_price = df_new_game['price'].isna().sum()

# Contar la cantidad de valores "None" en la columna "items_count"
count_none_items_count = df_new_game['items_count'].isna().sum()

# Imprimir los resultados
print(f'Cantidad de valores "None" en la columna "price": {count_none_price}')
print(f'Cantidad de valores "None" en la columna "items_count": {count_none_items_count}')

Cantidad de valores "None" en la columna "price": 0
Cantidad de valores "None" en la columna "items_count": 0


In [187]:
# Reemplazar los valores "None" en la columna "user_id" con "Sin dato"
df_new_game_copy['user_id'].fillna("Sin dato", inplace=True)

In [188]:
# Contar valores duplicados en el DataFrame 
duplicados_total = df_new_game_copy.duplicated().sum()

# Contar valores duplicados en la columna "user_id"
duplicados_en_user_id = df_new_game_copy['user_id'].duplicated().sum()
duplicados_en_recomendado = df_new_game_copy['items_count'].duplicated().sum()

print("Valores duplicados en todo el DataFrame:", duplicados_total)
print("Valores duplicados en la columna 'user_id':", duplicados_en_user_id)
print("Valores duplicados en la columna 'items_count':", duplicados_en_recomendado)

Valores duplicados en todo el DataFrame: 32674
Valores duplicados en la columna 'user_id': 32818
Valores duplicados en la columna 'items_count': 119520


In [189]:
#eliminar los duplicados
df_new_game_copy = df_new_game_copy.drop_duplicates(subset=['user_id'])

In [190]:
duplicados= df_new_game_copy.duplicated(['user_id']).sum()
print("Valores duplicados en user_id:", duplicados)


Valores duplicados en user_id: 0


In [191]:
df_new_game_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87627 entries, 0 to 88310
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   price        87627 non-null  float64
 1   items_count  87627 non-null  float64
 2   user_id      87627 non-null  object 
dtypes: float64(2), object(1)
memory usage: 2.7+ MB


In [192]:
# Ruta donde  guardar el archivo CSV
ruta_csv = r"C:\Users\rossa\Desktop\ProyectoIndividualML\ProyectoIndividualML\df_new_game_copy.csv"  

# Guarda el DataFrame en un archivo CSV
df_new_game_copy.to_csv(ruta_csv, index=False)  # El argumento 'index=False' evita que se incluya el índice en el archivo CSV

print("DataFrame guardado como archivo CSV en:", ruta_csv)

DataFrame guardado como archivo CSV en: C:\Users\rossa\Desktop\ProyectoIndividualML\ProyectoIndividualML\df_new_game_copy.csv


In [193]:
# Especifica la ruta completa del archivo CSV
ruta_csv = r"C:\Users\rossa\Desktop\ProyectoIndividualML\ProyectoIndividualML\df_new_game_copy.csv"

# Carga el archivo CSV en un DataFrame
df_new_game_copy.csv = pd.read_csv(ruta_csv)

C:\Users\rossa\AppData\Local\Temp\ipykernel_10212\3944001998.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_new_game_copy.csv = pd.read_csv(ruta_csv)


In [194]:
df_new_game_copy.head(2)

,price,items_count,user_id
0,0.0,277.0,76561197970982479
1,0.0,888.0,js41637


EXTRACCION COLUMNA GENZE PARA FUNCION 3 y 4

In [95]:
generos = data_df['genres']

generos

0                                          None
1                                          None
2                                          None
3                                          None
4                                          None
                          ...                  
120440    [Casual, Indie, Simulation, Strategy]
120441                [Casual, Indie, Strategy]
120442              [Indie, Racing, Simulation]
120443                          [Casual, Indie]
120444                                     None
Name: genres, Length: 120445, dtype: object

In [96]:
# Selecciona las columnas 'user_id', 'genres'
nuevo_game_df = data_df.loc[:, ['genres']]

#  nuevo_df contiene las columnas 'user_id', 'genres' 
nuevo_game_df

,genres
0,None
1,None
2,None
3,None
4,None
...,...
120440,"[Casual, Indie, Simulation, Strategy]"
120441,"[Casual, Indie, Strategy]"
120442,"[Indie, Racing, Simulation]"
120443,"[Casual, Indie]"


In [97]:
# mostrar las columnas de DF
nuevo_game_df.columns

Index(['genres'], dtype='object')

In [98]:
nuevo_game_df.sample(4)

,genres
70091,None
62372,None
42552,None
106361,[Strategy]


In [99]:
#Crea DF con columnas a utilizar - columna genres tiene datos anidados tipo lista

df_games_genero = data_df[['genres', 'app_name','user_id']]
df_games_genero

,genres,app_name,user_id
0,None,None,76561197970982479
1,None,None,js41637
2,None,None,evcentric
3,None,None,Riot-Punch
4,None,None,doctr
...,...,...,...
120440,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,None
120441,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,None
120442,"[Indie, Racing, Simulation]",Russian Roads,None
120443,"[Casual, Indie]",EXIT 2 - Directions,None


In [100]:
#filtrar las filas en las que la columna 'user_id' no sea nula

df = df_games_genero ['user_id'][df_games_genero ['user_id'].notna()]
type(df)


pandas.core.series.Series

In [101]:
 # Llenar los NaN en 'user_id' con los valores aleatorios de la lista
df_games_genero['user_id'].fillna(pd.Series(df), inplace=True)

C:\Users\rossa\AppData\Local\Temp\ipykernel_10212\2322240781.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_genero['user_id'].fillna(pd.Series(df), inplace=True)


In [102]:
df_games_genero['user_id'].fillna(pd.Series(df), inplace=True)

C:\Users\rossa\AppData\Local\Temp\ipykernel_10212\204334995.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_genero['user_id'].fillna(pd.Series(df), inplace=True)


In [103]:
df.isna().sum()

0

In [104]:
df_games_genero.sample(2)

,genres,app_name,user_id
90716,"[Simulation, Strategy]",Europa Universalis IV: Indian Subcontinent Uni...,NaN
34255,None,None,misslugosi


In [105]:
#Extraer los valores tipo lista de la columna genres a valores indiviuales
# la función str.join para combina los elementos de la columna 'genres' en una cadena separada por comas.
df_games_genero['genres'] = df_games_genero['genres'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

C:\Users\rossa\AppData\Local\Temp\ipykernel_10212\1782557317.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games_genero['genres'] = df_games_genero['genres'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)


In [106]:
# Se divide la columna 'genres' en columnas separadas para cada género utilizando str.get_dummies.
df_genero_div = df_games_genero['genres'].str.get_dummies(', ')

In [107]:
df_genero_div

,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,Free to Play,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
120441,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
120442,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
120443,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
# Concatenar las columnas generadas al DataFrame original.
df_games_genero = pd.concat([df_games_genero, df_genero_div], axis=1)

In [109]:
df_games_genero['genres'] = df_games_genero['genres'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [110]:
#genre está separada por comas
df_games_genero.sample(2)

,genres,app_name,user_id,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
96473,None,"Please, Don't Touch Anything 3D",NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3287,None,None,76561198067960427,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [111]:
#crear una columna por cada valor en la columna genres
# Itera a través de las columnas generadas a partir de 'genres' y crea una nueva columna para cada valor dividido.

for g_column in df_genero_div.columns:
        # Crea una nueva columna con el nombre del género y asigna 1 si ese género está presente en esa fila, 0 en caso contrario.
    df_games_genero[g_column] = df_genero_div[g_column]

    # Ahora tendrás una columna separada para cada valor dividido en las columnas generadas a partir de 'genres'.
    df_games_genero.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 25 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   genres                     28852 non-null   object
 1   app_name                   32133 non-null   object
 2   user_id                    88310 non-null   object
 3   Accounting                 120445 non-null  int64 
 4   Action                     120445 non-null  int64 
 5   Adventure                  120445 non-null  int64 
 6   Animation &amp; Modeling   120445 non-null  int64 
 7   Audio Production           120445 non-null  int64 
 8   Casual                     120445 non-null  int64 
 9   Design &amp; Illustration  120445 non-null  int64 
 10  Early Access               120445 non-null  int64 
 11  Education                  120445 non-null  int64 
 12  Free to Play               120445 non-null  int64 
 13  Indie                      120445 non-null  

In [112]:
df_games_genero.sample(2)

,genres,app_name,user_id,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
116945,Action,Call of Duty®: Ghosts - Hydra Pack,NaN,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97629,"Adventure, Casual, Indie, RPG, Strategy",Item Pack for Arcane RE-Raise,NaN,0,0,1,0,0,1,0,...,0,1,0,0,0,0,1,0,0,0


In [113]:
df_games_genero.isna().sum()

genres                       91593
app_name                     88312
user_id                      32135
Accounting                       0
Action                           0
Adventure                        0
Animation &amp; Modeling         0
Audio Production                 0
Casual                           0
Design &amp; Illustration        0
Early Access                     0
Education                        0
Free to Play                     0
Indie                            0
Massively Multiplayer            0
Photo Editing                    0
RPG                              0
Racing                           0
Simulation                       0
Software Training                0
Sports                           0
Strategy                         0
Utilities                        0
Video Production                 0
Web Publishing                   0
dtype: int64

In [114]:
df_games_genero.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 25 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   genres                     28852 non-null   object
 1   app_name                   32133 non-null   object
 2   user_id                    88310 non-null   object
 3   Accounting                 120445 non-null  int64 
 4   Action                     120445 non-null  int64 
 5   Adventure                  120445 non-null  int64 
 6   Animation &amp; Modeling   120445 non-null  int64 
 7   Audio Production           120445 non-null  int64 
 8   Casual                     120445 non-null  int64 
 9   Design &amp; Illustration  120445 non-null  int64 
 10  Early Access               120445 non-null  int64 
 11  Education                  120445 non-null  int64 
 12  Free to Play               120445 non-null  int64 
 13  Indie                      120445 non-null  

In [115]:
# Selección de columnas
columnas_necesarias = ['user_id','app_name','Action', 'Adventure', 'Racing', 'Simulation', 'Strategy','Sports']

    # Crear una lista de columnas seleccionadas
columnas_descartadas = [col for col in df_games_genero.columns if col not in columnas_necesarias]

    # Eliminar las columnas no necesarias
df_games_genero = df_games_genero.drop(columnas_descartadas, axis=1)

df_games_genero.sample(2)

,app_name,user_id,Action,Adventure,Racing,Simulation,Sports,Strategy
35048,None,76561198061879489,0,0,0,0,0,0
51073,None,Red_Kn1ght,0,0,0,0,0,0


In [116]:
# Crear una serie con los datos de la columna 'user_id' para rellenar los valores NaN
valores = df_games_genero['user_id'][df_games_genero['user_id'].notna()]

type(valores)

pandas.core.series.Series

In [117]:
valores

0              76561197970982479
1                        js41637
2                      evcentric
3                     Riot-Punch
4                          doctr
                  ...           
88305          76561198323066619
88306          76561198326700687
88307    XxLaughingJackClown77xX
88308          76561198329548331
88309          edward_tremethick
Name: user_id, Length: 88310, dtype: object

In [121]:
# funcion para rellenar los NaN de la columna

def completar_user_id(row):
    if pd.notna(row['app_name']) and pd.isna(row['user_id']):
            return np.random.choice(valores)
    else:
            return row['user_id']

In [122]:
# copia del DataFrame x advertencia
df_copia = df_games_genero.copy()

# Aplica la función para llenar los valores NaN en 'user_id'
df_copia['user_id'] = df_games_genero.apply(completar_user_id, axis=1)

In [123]:
df_copia.sample(2)

,app_name,user_id,Action,Adventure,Racing,Simulation,Sports,Strategy
74899,None,76561198082490745,0,0,0,0,0,0
109168,Dashing Dinos,76561197983793259,1,0,0,0,0,0


In [124]:
df_copia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   app_name    32133 non-null   object
 1   user_id     120443 non-null  object
 2   Action      120445 non-null  int64 
 3   Adventure   120445 non-null  int64 
 4   Racing      120445 non-null  int64 
 5   Simulation  120445 non-null  int64 
 6   Sports      120445 non-null  int64 
 7   Strategy    120445 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 7.4+ MB


In [125]:
# Transformar columnas: app_name y user_id a tipo numérico

numero =['Action', 'Adventure', 'Racing', 'Simulation', 'Strategy','Sports']

# Convierte las columnas especificadas a tipo numérico
df_copia [numero] = df_copia [numero].apply(pd.to_numeric, errors='coerce')

df_copia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   app_name    32133 non-null   object
 1   user_id     120443 non-null  object
 2   Action      120445 non-null  int64 
 3   Adventure   120445 non-null  int64 
 4   Racing      120445 non-null  int64 
 5   Simulation  120445 non-null  int64 
 6   Sports      120445 non-null  int64 
 7   Strategy    120445 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 7.4+ MB


In [126]:
df_copia.isnull().sum()

app_name      88312
user_id           2
Action            0
Adventure         0
Racing            0
Simulation        0
Sports            0
Strategy          0
dtype: int64

In [127]:
porcentaje = 0.5  
df_copia  = df_copia.sample(frac=porcentaje)
df_copia

,app_name,user_id,Action,Adventure,Racing,Simulation,Sports,Strategy
77033,None,76561198086127331,0,0,0,0,0,0
119528,Quake IV,76561198105011167,1,0,0,0,0,0
79925,None,76561198090950615,0,0,0,0,0,0
62948,None,76561198067254286,0,0,0,0,0,0
16183,None,76561198066514889,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
55702,None,76561198054683289,0,0,0,0,0,0
81112,None,76561198092774409,0,0,0,0,0,0
59599,None,76561198062524133,0,0,0,0,0,0
36979,None,76561198056006228,0,0,0,0,0,0


In [128]:
# Ruta donde se guarda el archivo CSV
ruta_csv = r"C:\Users\rossa\Desktop\ProyectoIndividualML\ProyectoIndividualML\df_games_genero3.csv"  

# Guarda el DataFrame en un archivo CSV
df_copia.to_csv(ruta_csv, index=False)  # El argumento 'index=False' evita que se incluya el índice en el archivo CSV

print("DataFrame guardado como archivo CSV en:", ruta_csv)

DataFrame guardado como archivo CSV en: C:\Users\rossa\Desktop\ProyectoIndividualML\ProyectoIndividualML\df_games_genero3.csv


In [129]:
#Carga de csv para fusionar los dos archivos

# Carga de archivo CSV df_games_genero3
ruta_csv = r"C:\Users\rossa\Desktop\ProyectoIndividualML\ProyectoIndividualML\df_games_genero3.csv"

# Carga el archivo CSV en un DataFrame
df_games_genero3 = pd.read_csv(ruta_csv)

In [130]:
#Carga de csv para fusionar los dos archivos

# Carga de archivo CSV df_items_ph3
ruta_csv = r"C:\Users\rossa\Desktop\ProyectoIndividualML\ProyectoIndividualML\df_items_ph3.csv"

# Carga el archivo CSV en un DataFrame
df_items_ph3 = pd.read_csv(ruta_csv)

In [131]:
## Fusionar los DataFrames en uno solo utilizando 'user_id' como clave
df_merged = df_items_ph3.merge(df_games_genero3, on='user_id', how='inner')

In [132]:
df_merged.sample(2)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_hours,app_name,Action,Adventure,Racing,Simulation,Sports,Strategy
113813,StuntMobile,733,76561198037517824,http://steamcommunity.com/id/StuntMobile,261030.0,The Walking Dead: Season Two,0.0,0.0,Fierce Tales: Marcus' Memory Collector's Edition,0,1,0,0,0,0
157073,76561198115099755,100,76561198115099755,http://steamcommunity.com/profiles/76561198115...,223530.0,Left 4 Dead 2 Beta,0.0,0.0,NaN,0,0,0,0,0,0


In [133]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362775 entries, 0 to 362774
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           362775 non-null  object 
 1   items_count       362775 non-null  int64  
 2   steam_id          362775 non-null  int64  
 3   user_url          362775 non-null  object 
 4   item_id           361557 non-null  float64
 5   item_name         361557 non-null  object 
 6   playtime_forever  362775 non-null  float64
 7   playtime_hours    362775 non-null  float64
 8   app_name          96199 non-null   object 
 9   Action            362775 non-null  int64  
 10  Adventure         362775 non-null  int64  
 11  Racing            362775 non-null  int64  
 12  Simulation        362775 non-null  int64  
 13  Sports            362775 non-null  int64  
 14  Strategy          362775 non-null  int64  
dtypes: float64(3), int64(8), object(4)
memory usage: 41.5+ MB


In [134]:
# contar los valores NaN en cada columna
nan_count = df_merged.isna().sum()

# Imprime el resultado
print(nan_count)

user_id                  0
items_count              0
steam_id                 0
user_url                 0
item_id               1218
item_name             1218
playtime_forever         0
playtime_hours           0
app_name            266576
Action                   0
Adventure                0
Racing                   0
Simulation               0
Sports                   0
Strategy                 0
dtype: int64


In [135]:
# Crea una máscara booleana que es True para las filas sin NaN y False para las filas con NaN
mask = df_merged.notna().all(axis=1)

# Selecciona las filas que cumplan con la máscara
df_merged = df_merged[mask]

In [136]:
nan_count = df_merged.isna().sum()

# Imprime el resultado
print(nan_count)

user_id             0
items_count         0
steam_id            0
user_url            0
item_id             0
item_name           0
playtime_forever    0
playtime_hours      0
app_name            0
Action              0
Adventure           0
Racing              0
Simulation          0
Sports              0
Strategy            0
dtype: int64


In [137]:
porcentaje = 0.5  
df_merged  = df_merged.sample(frac=porcentaje)
df_merged 

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_hours,app_name,Action,Adventure,Racing,Simulation,Sports,Strategy
31905,Taunch,104,76561198056795449,http://steamcommunity.com/id/Taunch,21110.0,F.E.A.R.: Extraction Point,0.0,0.00,Rocksmith® 2014 – Deftones Song Pack,0,0,0,1,0,0
46988,jarvisnumbanine,195,76561198116441756,http://steamcommunity.com/id/jarvisnumbanine,11390.0,Crash Time II,7.0,0.00,Disassembly Line,1,0,0,0,0,0
30808,Mediev,1018,76561198082166693,http://steamcommunity.com/id/Mediev,519574.0,Complete Figure Drawing Course HD: 015 - The J...,0.0,0.00,DOOM 3,1,0,0,0,0,0
231151,ayylmaokai420,79,76561198065776201,http://steamcommunity.com/id/ayylmaokai420,271290.0,HAWKEN,0.0,0.00,Chuck's Challenge 3D,0,0,0,0,0,0
110395,Praiseofdeph,515,76561198103889275,http://steamcommunity.com/id/Praiseofdeph,395500.0,Rot Gut,53.0,0.01,Defend Your Life: TD,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339381,JU5TCE,28,76561198047957987,http://steamcommunity.com/id/JU5TCE,218230.0,PlanetSide 2,93.0,0.03,Future Farmer,0,0,0,0,0,0
249001,76561198056997935,332,76561198056997935,http://steamcommunity.com/profiles/76561198056...,17390.0,Spore,1153.0,0.32,Vektor Wars,1,0,0,0,0,0
324080,henzo800,38,76561198139579706,http://steamcommunity.com/id/henzo800,365590.0,Tom Clancy's The Division,4724.0,1.31,Gulu,0,0,0,0,0,0
301881,Cyber_Z0,58,76561198042372402,http://steamcommunity.com/id/Cyber_Z0,8870.0,BioShock Infinite,283.0,0.08,St Christopher's School Lockdown - Soundtrack,0,1,0,0,0,0


In [20]:
df_fusion3 = pd.read_csv( r"C:\Users\rossa\Desktop\ProyectoIndividualML\ProyectoIndividualML\df_fusion3.csv")

In [21]:
df_fusion3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476956 entries, 0 to 476955
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           476956 non-null  object 
 1   items_count       476956 non-null  int64  
 2   steam_id          476956 non-null  int64  
 3   user_url          476956 non-null  object 
 4   item_id           476956 non-null  float64
 5   item_name         476956 non-null  object 
 6   playtime_forever  476956 non-null  float64
 7   playtime_hours    476956 non-null  float64
 8   app_name          476956 non-null  object 
 9   Action            476956 non-null  int64  
 10  Adventure         476956 non-null  int64  
 11  Racing            476956 non-null  int64  
 12  Simulation        476956 non-null  int64  
 13  Sports            476956 non-null  int64  
 14  Strategy          476956 non-null  int64  
dtypes: float64(3), int64(8), object(4)
memory usage: 54.6+ MB


In [22]:
porcentaje = 0.5  
df_merged  = df_fusion3.sample(frac=porcentaje)
df_merged

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_hours,app_name,Action,Adventure,Racing,Simulation,Sports,Strategy
351018,vincentsu88,42,76561198108222756,http://steamcommunity.com/id/vincentsu88,317360.0,Double Action: Boogaloo,127.0,0.04,Tupã,1,1,0,0,0,0
117481,76561198070585472,4456,76561198070585472,http://steamcommunity.com/profiles/76561198070...,39680.0,The Guild II: Renaissance,0.0,0.00,Epic Mayhem,1,0,1,0,1,0
456219,76561198014421677,63,76561198014421677,http://steamcommunity.com/profiles/76561198014...,20540.0,Company of Heroes: Tales of Valor,1070.0,0.30,Street Fighter X Tekken: Additional 12 Charact...,1,0,0,0,0,0
217049,76561198040615827,148,76561198040615827,http://steamcommunity.com/profiles/76561198040...,105450.0,Age of Empires® III: Complete Collection,1880.0,0.52,Rocksmith® 2014 – Biffy Clyro - “Mountains”,0,0,0,1,0,0
45718,76561198018589800,380,76561198018589800,http://steamcommunity.com/profiles/76561198018...,65980.0,Sid Meier's Civilization: Beyond Earth,479.0,0.13,Gnomelings: Migration,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39133,76561198044882472,326,76561198044882472,http://steamcommunity.com/profiles/76561198044...,203160.0,Tomb Raider,0.0,0.00,AIPD - Soundtrack,1,0,0,0,0,0
430830,FURYZ619,379,76561198019110263,http://steamcommunity.com/id/FURYZ619,238590.0,Loadout Campaign Beta,0.0,0.00,Whack-a-Vote: Hammering the Polls,0,0,0,0,0,0
304334,76561197985797191,2287,76561197985797191,http://steamcommunity.com/profiles/76561197985...,244870.0,Electronic Super Joy,0.0,0.00,Haywire,0,0,0,0,0,0
375805,LeZoidberg,1304,76561197961112434,http://steamcommunity.com/id/LeZoidberg,55100.0,Homefront,0.0,0.00,The Legacy Games Demo,0,0,0,0,0,1


In [23]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 238478 entries, 351018 to 85758
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           238478 non-null  object 
 1   items_count       238478 non-null  int64  
 2   steam_id          238478 non-null  int64  
 3   user_url          238478 non-null  object 
 4   item_id           238478 non-null  float64
 5   item_name         238478 non-null  object 
 6   playtime_forever  238478 non-null  float64
 7   playtime_hours    238478 non-null  float64
 8   app_name          238478 non-null  object 
 9   Action            238478 non-null  int64  
 10  Adventure         238478 non-null  int64  
 11  Racing            238478 non-null  int64  
 12  Simulation        238478 non-null  int64  
 13  Sports            238478 non-null  int64  
 14  Strategy          238478 non-null  int64  
dtypes: float64(3), int64(8), object(4)
memory usage: 29.1+ MB


In [24]:
# Ruta donde se guarda el archivo CSV
ruta_csv = r"C:\Users\rossa\Desktop\ProyectoIndividualML\ProyectoIndividualML\df_fusion3.csv"  

# Guarda el DataFrame en un archivo CSV
df_merged.to_csv(ruta_csv, index=False)  

print("DataFrame guardado como archivo CSV en:", ruta_csv)

DataFrame guardado como archivo CSV en: C:\Users\rossa\Desktop\ProyectoIndividualML\ProyectoIndividualML\df_fusion3.csv
